In [2]:
%pip install numpy scikit-learn requests

   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ------ --------------------------------- 10.2/62.6 kB ? eta -:--:--
   ------------------- -------------------- 30.7/62.6 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------  61.4/62.6 kB 544.7 kB/s eta 0:00:01
   ---------------------------------------- 62.6/62.6 kB 475.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
   ------- -------------------------------- 30.7/163.8 kB 1.3 MB/s eta 0:00:01
   ----------------- --------------------- 71.7/163.8 kB 975.2 kB/s eta 0:00:01
   -------------------------- ----------- 112.6/163.8 kB 930.9 kB/s eta 0:00:01
   -------------------------- ----------- 112.6/163.8 kB 930.9 kB/s eta 0:00:01
   -------------------------- ----------- 112.6/163.8 kB 930.9 kB/s eta 0:00:01
   ---------------------------- --------- 122.9/163.8 kB 423.5 kB/s eta 0:00:01
   -------------------------------------- 163.8/163.8 kB 516.9 kB/s e

In [20]:
import numpy as np
import requests
import pandas as pd

response= requests.get("https://data.cityofnewyork.us/resource/2bnn-yakx.json")
response_json= response.json()
df= pd.DataFrame(response_json)


    summons_number plate_id registration_state plate_type  \
0       5092469481  GZH7067                 NY        PAS   
1       5092451658  GZH7067                 NY        PAS   
2       4631633384  AVM7975                 NY        PAS   
3       8196557280  GWB7054                 NY        PAS   
4       4631184358  EXZ9820                 NY        PAS   
..             ...      ...                ...        ...   
995     8407013183  GKR2178                 NY        PAS   
996     8480008386   AN251V                 NJ        PAS   
997     7074247583  HAV1171                 NY        PAS   
998     8206015632  42268PC                 NY        IRP   
999     7924512226  HNK2852                 PA        PAS   

                  issue_date violation_code vehicle_body_type vehicle_make  \
0    2016-07-10T00:00:00.000              7              SUBN        TOYOT   
1    2016-07-08T00:00:00.000              7              SUBN        TOYOT   
2    2017-03-09T00:00:00.000     